In [1]:
import pandas as pd
import os
import numpy as np

Date Culomn: 1
Time Culomn: 2
CO Culomn: 4
SO2 Culomn: 8
PM10 Culomn: 9

In [2]:
# Set the directory containing the Excel files
directory = 'Dataset/'

# Create an empty list to store the dataframes
dfs = []
index = 0
# Loop through each file in the directory from seocnd row
for filename in os.listdir(directory):
    
    if filename.endswith('.xlsx'):
        # Read the data from the Excel file
        data = pd.read_excel(os.path.join(directory, filename))

        # cast to float
        data.iloc[1:, [4,8,9]] = data.iloc[1:, [4,8,9]].astype(float)


        # add index to column 0
        data.iloc[0] = [index for j in range(data.shape[1])]
        index += 1

        print(index, ')',filename, data.shape[0])

        # Add the dataframe from second row to the list
        # dfs.append(data.iloc[2:, [0,1,4,8,9]])
        dfs.append(pd.DataFrame(data={'date': data.iloc[1:, 1],
                    'time': data.iloc[1:, 2],
                    'co': data.iloc[1:, 4],
                    'so2': data.iloc[1:, 8],
                    'pm10': data.iloc[1:, 9]}, index = [i for i in range(data.shape[0] - 1)]))

for i in range(len(dfs)):
    dfs[i] = dfs[i][1:]

1 ) شادآباد.منطقه18.xlsx 17442
2 ) ستاد.بحران.منطقه7.xlsx 17393
3 ) پونک.xlsx 17486
4 ) پیروزی.منطقه13.xlsx 16953
5 ) میدان.فتح.منطقه9.xlsx 17184
6 ) شهرداری.منطقه21.xlsx 17389
7 ) شهرداری.منطقه22.xlsx 17145
8 ) اتوبان.محلاتی.منطقه14.xlsx 17073
9 ) تربیت.مدرس.منطقه6.xlsx 17356
10 ) شهرداری.منطقه.2.xlsx 17457
11 ) اقدسیه.منطقه1.xlsx 17435
12 ) مسعودیه.منطقه15.xlsx 17226
13 ) شهر.ری.منطقه20.xlsx 17265


In [3]:
print(dfs[0])
# print(dfs[0][dfs[0][1] == '1400/01/01' & dfs[0][2] == '0'])

             date time        co       so2       pm10
1      1400/01/01    0  1.187312    3.2108       45.5
2      1400/01/01    2  1.371816  3.274771  47.399814
3      1400/01/01    3  1.021523  3.434897  36.807692
4      1400/01/01    4  1.139674  3.843559  20.463772
5      1400/01/01    5  1.109617  3.235971  32.748563
...           ...  ...       ...       ...        ...
17436  1401/12/29   18       NaN  2.596993  20.140805
17437  1401/12/29   19       NaN  3.119621  53.106667
17438  1401/12/29   20       NaN  5.027132        NaN
17439  1401/12/29   21       NaN  5.325556        NaN
17440  1401/12/29   22       NaN  5.794776  47.086207

[17440 rows x 5 columns]


In [21]:
# date
date = '1401/01/04'
hour = 0
i = 2

print(dfs[i][(dfs[i]['date'] == date) & (dfs[i]['time'] == hour)].shape[0] > 0)

# # check if date and hour exist in dfs[i] in same row
# if dfs[i][dfs[i].date == date & dfs[i].time == hour].shape[0] > 0:
#     print('yes')
#     # get index
#     index = dfs[1].index[(dfs[i].iloc[:, 1] == date) & (dfs[i].iloc[:, 2] == hour)].tolist()[0]
#     print(index)
#     print(dfs[i].iloc[index - 1])
#     # dfs[1].drop(index, inplace=True)
#     # print(dfs[1].iloc[index - 1])
# else:
#     print('no')

True


In [4]:
# Cleaning the data
years = [1400, 1401]
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = [31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 29]
hours = [i for i in range(24)]

prv_date = '1400/01/01'
prv_time = 0
column_names = dfs[0].columns
print(column_names)
new_dfs = [pd.DataFrame(columns=column_names) for i in range(len(dfs))]

for year in years: 
    for max_day, month in zip(days, months):
        for day in range(1, max_day + 1):
            if day < 10:
                day = f'0{day}'
            cur_date = f'{year}/{month}/{day}'
            for cur_hour in hours:
                for i in range(len(dfs)):
                    row = dfs[i][(dfs[i]['date'] == cur_date) & (dfs[i]['time'] == cur_hour)] 
                    if row.shape[0] > 0:
                        # get index
                        # print('yes', cur_date, cur_hour, i)
                        new_row = {'date': cur_date, 
                                   'time': cur_hour, 
                                   'co': row['co'].values[0], 
                                   'so2': row['so2'].values[0], 
                                   'pm10': row['pm10'].values[0]}
                        new_dfs[i].loc[len(new_dfs[i])] = new_row
                        
                    else:
                        # add null row
                        new_dfs[i].loc[len(new_dfs[i])] = {'date': cur_date,
                                                           'time': cur_hour,
                                                           'co': np.NaN,
                                                           'so2': np.NaN,
                                                           'pm10': np.NaN}
                        
                prv_time = cur_hour
            prv_date = cur_date
            # print(new_dfs[0])
        print(f'{year}/{month} done')
            
print(new_dfs[0])

# Save the dataframes to csv files
for i in range(len(new_dfs)):
    new_dfs[i].to_csv(f'Clean with NaN/cleaned_data_{i}.csv', index=False)
                        

Index(['date', 'time', 'co', 'so2', 'pm10'], dtype='object')


KeyboardInterrupt: 

In [32]:
# read from csv
new_dfs = []
for i in range(len(dfs)):
    new_dfs.append(pd.read_csv(f'Clean with NaN/cleaned_data_{i}.csv'))

In [6]:
# print shape of each dataframe
for i in range(len(new_dfs)):
    print(new_dfs[i].shape)

(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)
(17520, 5)


In [31]:
import numpy as np
# for null values: make average of other values in same day and hour

row_num = new_dfs[0].shape[0]
col_num = new_dfs[0].shape[1]
node_num = len(new_dfs)

# print(row_num, node_num)

# iterate on rows of every dataframe
for i in range(row_num):
    all_rows = []

    # iterate on dataframes
    for j in range(node_num):
        all_rows.append(new_dfs[j].iloc[i].to_list())
    
    # print(f'{i}) ** ', all_rows)

    co = np.nanmean([all_rows[k][2] for k in range(node_num)])
    so2 = np.nanmean([all_rows[k][3] for k in range(node_num)])
    pm10 = np.nanmean([all_rows[k][4] for k in range(node_num)])

    
    # print(f'{i})', co, so2, pm10)

    # fill the null values with 0.7 * average + 0.3 * previous value
    for j in range(node_num):
        # co2 
        if np.isnan(all_rows[j][2]):
            if np.isnan(co):
                new_dfs[j].iloc[i, 2] = new_dfs[j].iloc[i - 1, 2]
            else:
                new_dfs[j].iloc[i, 2] = 0.7 * co + 0.3 * new_dfs[j].iloc[i - 1, 2]

        # so2
        if np.isnan(all_rows[j][3]):
            if np.isnan(so2):
                new_dfs[j].iloc[i, 3] = new_dfs[j].iloc[i - 1, 3]
            else:
                new_dfs[j].iloc[i, 3] = 0.7 * so2 + 0.3 * new_dfs[j].iloc[i - 1, 3]

        # pm10
        if np.isnan(all_rows[j][4]):
            if np.isnan(pm10):
                new_dfs[j].iloc[i, 4] = new_dfs[j].iloc[i - 1, 4]
            else:
                new_dfs[j].iloc[i, 4] = 0.7 * pm10 + 0.3 * new_dfs[j].iloc[i - 1, 4]
        
    # if i == 3:
    #     break
    if i % 500 == 0:
        print(f'{i} rows done')

print(new_dfs[0])

# Save the dataframes to csv files
for i in range(len(new_dfs)):

    new_dfs[i].to_csv(f'Clean/cleaned_data_{i}.csv', index=False)


0 rows done
500 rows done
1000 rows done
1500 rows done
2000 rows done
2500 rows done
3000 rows done
3500 rows done
4000 rows done
4500 rows done
5000 rows done
5500 rows done
6000 rows done
6500 rows done
7000 rows done
7500 rows done
8000 rows done
8500 rows done


/var/folders/h8/mhwh4fjn0qj405sjdd9_5ffw0000gn/T/ipykernel_22646/1167662963.py:20: RuntimeWarning: Mean of empty slice
  co = np.nanmean([all_rows[k][2] for k in range(node_num)])
/var/folders/h8/mhwh4fjn0qj405sjdd9_5ffw0000gn/T/ipykernel_22646/1167662963.py:21: RuntimeWarning: Mean of empty slice
  so2 = np.nanmean([all_rows[k][3] for k in range(node_num)])
/var/folders/h8/mhwh4fjn0qj405sjdd9_5ffw0000gn/T/ipykernel_22646/1167662963.py:22: RuntimeWarning: Mean of empty slice
  pm10 = np.nanmean([all_rows[k][4] for k in range(node_num)])


9000 rows done
9500 rows done
10000 rows done
10500 rows done
11000 rows done
11500 rows done
12000 rows done
12500 rows done
13000 rows done
13500 rows done
14000 rows done
14500 rows done
15000 rows done
15500 rows done
16000 rows done
16500 rows done
17000 rows done
17500 rows done
             date  time        co       so2       pm10
0      1400/01/01     0  1.187312  3.210800  45.500000
1      1400/01/01     1  1.187312  3.210800  45.500000
2      1400/01/01     2  1.371816  3.274771  47.399814
3      1400/01/01     3  1.021523  3.434897  36.807692
4      1400/01/01     4  1.139674  3.843559  20.463772
...           ...   ...       ...       ...        ...
17515  1401/12/29    19  0.954321  3.119621  53.106667
17516  1401/12/29    20  1.258243  5.027132  37.154505
17517  1401/12/29    21  1.410335  5.325556  37.341226
17518  1401/12/29    22  1.508488  5.794776  47.086207
17519  1401/12/29    23  1.508488  5.794776  47.086207

[17520 rows x 5 columns]
